In [3]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import pandas
import time
import os
import xlwt
from openpyxl import load_workbook
from shutil import move
from datetime import datetime, timedelta
from selenium.webdriver.support.ui import WebDriverWait

In [4]:
url="https://www.amazon.com/-/zh_TW/s?i=electronics-intl-ship&bbn=16225009011&rh=n%3A541966%2Cn%3A193870011%2Cn%3A17923671011%2Cn%3A284822&dc&page={}&language=zh_TW&qid=1598425548&rnid=17923671011&ref=sr_pg_{}"
options = Options()
options.add_argument("--disable-notifications")
options.add_argument('headless')
chrome = webdriver.Chrome('./chromedriver')
amazon_allpage = []
data = []
tStart = time.time()
for i in range(1,3):
    chrome.get(url.format(i,i))
    soup = BeautifulSoup(chrome.page_source,'lxml')
    amazon_card=soup.find_all(class_="a-size-medium a-color-base a-text-normal")
    if len(amazon_card) == 0:
        print("找不到顯示卡標題")
        break
    data.append([ele.text for ele in amazon_card])    
chrome.close()    
tEnd = time.time()
print("總共耗時:%f 秒" % (tEnd-tStart))

總共耗時:8.074801 秒


In [15]:
amazon_json = json.load(open("AMAZON_COMPUTER_PRICE" + ".json",encoding="utf-8"))
df = pd.DataFrame(data)
df = df.T
df.rename(columns={0:'第一頁',1:'第二頁'},inplace=True)
print(df)

                                                  第一頁  \
0                                         XFX HDMI 顯卡   
1   XFX Thicc Boost Up Graphics Card 內建式 纖維 10000M...   
2   Gigabyte 技嘉 Gv-R57XTGAMING OC-8GD Radeon Rx 57...   
3                                         XFX HDMI 顯卡   
4   MSI Gaming GeForce RTX 2060 6GB GDRR6 192-bit ...   
5   XFX RX 5700 XT 三重耗 8GB GDDR6 3xDP HDMI 顯示卡 RX-...   
6   Sapphire 11265-05-20G Radeon Pulse RX 580，8GB，...   
7   EVGA GeForce RTX 2060 Super XC Gaming, 8GB GDD...   
8   PowerColor Red Devil Radeon RX 5700 XT 8GB GDD...   
9   Sapphire 11293-01-20G Radeon PULSE RX 5700 Xt ...   
10  Gigabyte 技嘉 Gv-N1660OC-6GD GeForce GTX 1660 OC...   
11  MSI Gaming GeForce GTX 1660 192-Bit HDMI/DP 6G...   
12  GIGABYTE GeForce GTX 1660 Ti OC 6G 192-bit GDD...   
13  Sapphire 11293-03-40G Radeon NITRO+ RX 5700 XT...   
14  MSI微星 Gaming GeForce GT 710 2GB GDRR3 64 位元 HD...   
15  EVGA GeForce RTX 2080 Super Gaming LGA 775 (So...   

                              

In [19]:
now_date = datetime.strftime(datetime.now() , '%Y%m%d')
# 沒檔案的話則建立
output_file = now_date + "_amazon_graphics card.xlsx"
target_path = "E:\\python\\比價、受歡迎系統\\Amazon\\Amazon_graphics_card\\"
if os.path.exists(output_file) == False:
    writer = pd.ExcelWriter(output_file, engine='openpyxl')
    writer.book.create_sheet('default')  # 新增空sheet
else:  # 如果檔案存在，append
    book = load_workbook(output_file)
    writer = pd.ExcelWriter(output_file, engine='openpyxl')
    writer.book = book
# 假如excel已經有json的sheet_name則移除excel的sheet
if amazon_json['sheet_name'] in writer.book.sheetnames:
    writer.book.remove(writer.book[amazon_json['sheet_name']])


# 將資料寫入excel
df.to_excel(writer, sheet_name=amazon_json['sheet_name'], index=False) 

if (len(writer.book.sheetnames) > 1):  # 若有其他sheet，刪除default sheet
    if 'default' in writer.book.sheetnames:
        writer.book.remove(writer.book['default'])    
writer.save()  


TypeError: unhashable type: 'list'

In [20]:
amazon_json['sheet_name'][0]

'顯示卡'

In [12]:
cwd = os.getcwd()
curr_file = cwd +"\\"+ output_file
target_file = target_path + output_file
try:
    move(curr_file, target_file)
    print(output_file + "\n移動檔案完畢.")
except Exception as e:
    print(output_file + "\n移動檔案失敗.")
    print(e.args)

20200901_amazon_graphics card.xlsx
移動檔案完畢.
